<a href="https://colab.research.google.com/github/jada9879/docker/blob/main/docs/pinecone-quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/pinecone-quickstart.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/pinecone-quickstart.ipynb)

# Pinecone Database quickstart

This notebook shows you how to set up and use Pinecone Database for high-performance similarity search.

## Install an SDK

Pinecone provides [SDKs](https://docs.pinecone.io/reference/pinecone-sdks) in multiple languages.

For this quickstart, install the Python SDK and a library that makes it easy to sign up with Pinecone:

In [8]:
!pip install -qU \
    pinecone \
    pinecone-notebooks

## Get an API key

You need an API key to make calls to your Pinecone project.

Use the widget below to generate a key. If you don't have a Pinecone account, the widget will sign you up for the free Starter plan.

In [9]:
import os

if not os.environ.get("pcsk_71qQmY_***********"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

## Initialize a client

Use the generated API key to intialize a client connection to Pinecone:

In [10]:
from pinecone import Pinecone, ServerlessSpec

api_key = "pcsk_71qQmY_***********"





## Generate vectors

A [vector embedding](https://www.pinecone.io/learn/vector-embeddings/) is a numerical representation of data that enables similarity-based search in vector databases like Pinecone. To convert data into this format, you use an embedding model.

For this quickstart, use the [`multilingual-e5-large`](https://docs.pinecone.io/models/multilingual-e5-large) embedding model hosted by Pinecone to [convert](https://docs.pinecone.io/guides/inference/generate-embeddings) four sentences about apples into vectors, three related to health, one related to cultivation.

In [11]:

api_key = "pcsk_71qQmY_***********"





## Create an index

In Pinecone, you store data in an [index](https://docs.pinecone.io/guides/indexes/understanding-indexes).

Create a serverless index that matches the dimension (`1024`) and similarity metric (`cosine`) of the `multilingual-e5-large` model you used in the previous step, and choose a [cloud and region](https://docs.pinecone.io/guides/indexes/understanding-indexes#cloud-regions) for hosting the index:

In [12]:
index_name = "dense-index"

In [36]:

import os
import time
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
        api_key=os.environ.get("pcsk_2ERwVD_***********")
    )


if index_name not in pc.list_indexes().names():
  spec = ServerlessSpec(cloud="aws", region="us-east-1")
  pc.create_index(
        index_name,
        dimension=1536, # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec # Ensure 'spec' is defined appropriately before this line
    )

  time.sleep(1)







## Upsert vectors

Target your index and use the [`upsert`](https://docs.pinecone.io/guides/data/upsert-data) operation to load your vector embeddings into a new namespace.

**Note:** [Namespaces](https://docs.pinecone.io/guides/get-started/key-features#namespaces) let you partition records within an index and are essential for [implementing multitenancy](https://docs.pinecone.io/guides/get-started/implement-multitenancy) when you need to isolate the data of each customer/user.


In [43]:

import os
import time
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key=os.environ.get("pcsk_2ERwVD_***********")
)

index_name = "dense-index"

if index_name not in pc.list_indexes().names():
    # Create a serverless index with integrated inference and sparse values enabled.
    spec = ServerlessSpec(cloud="aws", region="us-east-1", sparse_values_enabled=True, inference_config={"pod_type": "p1"})
    pc.create_index(
        index_name,
        dimension=1536,
        metric="cosine",
        spec=spec
    )
    time.sleep(1)

**Note:** To load large amounts of data, [import from object storage](https://docs.pinecone.io/guides/data/understanding-imports) or [upsert in large batches](https://docs.pinecone.io/guides/data/upsert-data#upsert-records-in-batches).

## Check the index

Pinecone is eventually consistent, so there can be a delay before your upserted vectors are available to query. Use the [`describe_index_stats`](https://docs.pinecone.io/guides/data/data-freshness/check-data-freshness) operation to check if the current vector count matches the number of vectors you upserted:

In [44]:
time.sleep(10)  # Wait for the upserted vectors to be indexed

print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


## Search the index

Now, let’s say you want to search your index for information related to "health risks".

Use the the `multilingual-e5-large` model hosted by Pinecone *to* convert your query into a vector embedding, and then use the [`query`](https://docs.pinecone.io/guides/data/query-data) operation to search for the three vectors in the index that are most semantically similar to the query vector:

In [45]:
# Define your query
query = "Health risks"

# Convert the query into a numerical vector that Pinecone can search with
query_embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

# Search the index for the three most similar vectors
results = index.query(
    namespace="example-namespace",
    vector=query_embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [], 'namespace': 'example-namespace', 'usage': {'read_units': 1}}


Notice that the response includes only records related to health, not the cultivation of apple.

## Add reranking

You can increase the accuracy of your search by reranking results based on their relevance to the query.

Use the `rerank` operation and the `bge-reranker-v2-m3` reranking model hosted by Pinecone to rerank the values of the documents.source_text fields:

In [46]:
# Rerank the search results based on their relevance to the query
ranked_results = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query="Disease prevention",
    documents=[
        {"id": "rec3", "source_text": "Rich in vitamin C and other antioxidants, apples contribute to immune health and may reduce the risk of chronic diseases."},
        {"id": "rec1", "source_text": "Apples are a great source of dietary fiber, which supports digestion and helps maintain a healthy gut."},
        {"id": "rec4", "source_text": "The high fiber content in apples can also help regulate blood sugar levels, making them a favorable snack for people with diabetes."}
    ],
    top_n=3,
    rank_fields=["source_text"],
    return_documents=True,
    parameters={
        "truncate": "END"
    }
)

print(ranked_results)


RerankResult(
  model='bge-reranker-v2-m3',
  data=[{
    index=0,
    score=0.13683891,
    document={
        id='rec3',
        source_text='Rich in vitamin C and other antioxidants, apples contribute to immune health and may reduce the risk of chronic diseases.'
    }
  },{
    index=1,
    score=0.0023596608,
    document={
        id='rec1',
        source_text='Apples are a great source of dietary fiber, which supports digestion and helps maintain a healthy gut.'
    }
  },{
    index=2,
    score=0.00084264233,
    document={
        id='rec4',
        source_text='The high fiber content in apples can also help regulate blood sugar levels, making them a favorable snack for people with diabetes.'
    }
  }],
  usage={'rerank_units': 1}
)


Notice that the two records specifically related to "health risks" (chronic disease and diabetes) are now ranked highest.

## Add filtering

You can use a [metadata filter](https://docs.pinecone.io/guides/data/understanding-metadata) to limit your search to records matching a filter expression.

Your upserted records contain a `category` metadata field. Now use that field as a filter to search for records in the “digestive system” category:

In [47]:
# Search the index with a metadata filter
filtered_results = index.query(
    namespace="example-namespace",
    vector=query_embedding.data[0].values,
    filter={
        "category": {"$eq": "digestive system"}
    },
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(filtered_results)

{'matches': [], 'namespace': 'example-namespace', 'usage': {'read_units': 1}}


Notice that the response includes only the one record in the “digestive system” category.

## Clean up

When you no longer need the `docs-quickstart-notebook` index, use the [`delete_index`](https://docs.pinecone.io/reference/api/control-plane/delete_index) operation to delete it:

In [48]:
pc.delete_index(index_name)

After you delete an index, you cannot use it again or recover it.